In [1]:
import os
import pandas as pd
from openpyxl import load_workbook

In [2]:
# Đường dẫn file Excel nguồn và file CSV xuất ra

source_path = r"C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Source\metadata_R8781_20251225 (1).xlsx"  # Thay bằng file thực tế

template_path = r"C:\Users\VUDUCHAI\Downloads\MGI Primer Updated 291025 (2).xlsx" # Thay bằng file thực tế

output_path = r"C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Output"  # Thư mục xuất CSV

csv_path = os.path.join(output_path, "template_barcode.csv")

print("File nguồn:", source_path)
print("File template:", template_path)
print("Thư mục xuất CSV:", output_path)
print("File CSV sẽ tạo:", csv_path)

File nguồn: C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Source\metadata_R8781_20251225 (1).xlsx
File template: C:\Users\VUDUCHAI\Downloads\MGI Primer Updated 291025 (2).xlsx
Thư mục xuất CSV: C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Output
File CSV sẽ tạo: C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Output\template_barcode.csv


In [3]:
# Mở file Excel nguồn từ biến source_path đã khai báo ở cell trên
wb_source = load_workbook(source_path, data_only=True)
print("Đã mở file nguồn:", source_path)
print("Sheet có trong file:", wb_source.sheetnames)

# In ra cột T từ dòng 22 của sheet đầu tiên hoặc sheet 'Sample'
sheet_name = wb_source.sheetnames[0]  # hoặc thay bằng 'Sample' nếu cần
ws = wb_source[sheet_name]

# Tạo DataFrame chứa các giá trị cột T từ dòng 22
values = []
row = 22
while True:
    value = ws[f"T{row}"].value
    if value is None:
        break
    values.append(value)
    row += 1
import pandas as pd
df_T = pd.DataFrame(values, columns=["T"])
display(df_T)
print(f"Tổng số dòng dữ liệu cột T: {len(df_T)}")

Đã mở file nguồn: C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Source\metadata_R8781_20251225 (1).xlsx
Sheet có trong file: ['Sample', 'SampleImport', 'Aviti Manifest', 'Aviti Manifest TEST', 'Definitions', 'Index Sequence', 'Index Sets', 'PCR']


,T
0,961
1,962
2,963
3,964
4,965
...,...
337,825
338,805
339,806
340,807


Tổng số dòng dữ liệu cột T: 342


In [4]:
# Xóa toàn bộ dữ liệu từ ô A2 trở đi trong sheet 'MỒI T7' của file template
wb_template = load_workbook(template_path)
ws_moi = wb_template["MỒI T7"]

row = 2
count = 0
while ws_moi[f"A{row}"].value is not None:
    ws_moi[f"A{row}"].value = None
    count += 1
    row += 1

wb_template.save(template_path)
print(f"Đã xóa {count} ô ở cột A từ dòng 2 trong sheet 'MỒI T7' và lưu lại file template.")

Đã xóa 0 ô ở cột A từ dòng 2 trong sheet 'MỒI T7' và lưu lại file template.


In [5]:
# Tạo DataFrame với cột đầu là df_T, cột 2 và 3 tính như VLOOKUP trong Excel
import pandas as pd
from openpyxl import load_workbook

# Giả sử df_T đã tồn tại (cột đầu là giá trị cần tra cứu)
# Đọc sheet 'PRIMER LOCKED' từ file template
wb = load_workbook(template_path, data_only=True)
ws_primer = wb["PRIMER LOCKED"]

# Đọc dữ liệu sheet 'PRIMER LOCKED' thành DataFrame
rows_primer = list(ws_primer.iter_rows(values_only=True))
max_col_primer = max(len(row) for row in rows_primer if row)
rows_primer = [row[:max_col_primer] for row in rows_primer]
df_primer = pd.DataFrame(rows_primer)

# Đặt tên cột cho df_primer nếu có header ở dòng 0
if not df_primer.empty:
    df_primer.columns = [str(col) for col in df_primer.iloc[0]]
    df_primer = df_primer[1:].reset_index(drop=True)

# Tạo DataFrame kết quả
result_df = pd.DataFrame()
result_df['A'] = df_T['T']  # Cột đầu tiên là df_T

# Cột 2: VLOOKUP(A2,'PRIMER LOCKED'!$A$2:$C$3000,2,FALSE)
lookup_dict_B = dict(zip(df_primer.iloc[:,0], df_primer.iloc[:,1]))
result_df['B'] = result_df['A'].map(lookup_dict_B)

# Cột 3: VLOOKUP(B2,'PRIMER LOCKED'!$B$2:$D$3000,2,FALSE)
lookup_dict_C = dict(zip(df_primer.iloc[:,1], df_primer.iloc[:,2]))
result_df['C'] = result_df['B'].map(lookup_dict_C)

# Hiển thị kết quả
display(result_df)
wb.close()

,A,B,C
0,961,S95193-S97193,ACCTTCTCCTTCGTTC
1,962,S95194-S97194,TCGTGGATGTCTAGGT
2,963,S95195-S97195,GTTCATGGACGTCGTA
3,964,S95196-S97196,TAGGATGCGAGCTCAA
4,965,S95197-S97197,CATGGAACCGTGTACT
...,...,...,...
337,825,S95057-S97057,GAAGGAAGATGACGTC
338,805,S95037-S97037,GATAGGCTTGTGACTG
339,806,S95038-S97038,AGTGGATCGTCATCGA
340,807,S95039-S97039,TTGGACGTAGCACTTC


In [6]:
# Tạo file CSV với ba dòng đầu như hình minh họa và dán result_df từ dòng 4, tên file có ngày tháng năm hiện tại
import os
import csv
from datetime import datetime

rows = [
    ['#barcodeName', 'T72_C_R8772_MGI_25_12'],
    ['#misMatch1', 0],
    ['#misMatch2', 0]
]

# Lấy ngày tháng năm hiện tại định dạng YYYYMMDD
current_date = datetime.now().strftime('%Y%m%d')
csv_filename = f'barcode_header_{current_date}.csv'
csv_path = os.path.join(output_path, csv_filename)

# Ghi 3 dòng đầu và dán result_df từ dòng 4
with open(csv_path, 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)
    result_df.to_csv(f, index=False, header=False)

print(f"Đã tạo file CSV với 3 dòng đầu và dán result_df từ dòng 4 tại: {csv_path}")
with open(csv_path, 'r', encoding='utf-8-sig') as f:
    print(f.read())

Đã tạo file CSV với 3 dòng đầu và dán result_df từ dòng 4 tại: C:\Users\VUDUCHAI\OneDrive - Gene Solutions\Tool Excel Team Bi\Metadata_tool\Output\barcode_header_20251231.csv
#barcodeName,T72_C_R8772_MGI_25_12
#misMatch1,0
#misMatch2,0
961,S95193-S97193,ACCTTCTCCTTCGTTC
962,S95194-S97194,TCGTGGATGTCTAGGT
963,S95195-S97195,GTTCATGGACGTCGTA
964,S95196-S97196,TAGGATGCGAGCTCAA
965,S95197-S97197,CATGGAACCGTGTACT
966,S95198-S97198,GCTTAGCTCACTGACA
967,S95199-S97199,CTAACTCGTCGTAGTC
968,S95200-S97200,ACCATGTGGCACGTAA
969,S95201-S97201,TCAGACGACAAGCAGT
970,S95202-S97202,TATCAGCGACATAGGC
971,S95203-S97203,AGCAGATGTGTGGTAC
972,S95204-S97204,AACGGTCACACCACTA
973,S95205-S97205,CGAACTGTCTGCGTAT
974,S95206-S97206,TCCGAGTTACGGTCTT
975,S95207-S97207,TTCTCTCGGATTGGAG
976,S95208-S97208,ATTCTGGCTGTCCAGA
977,S95209-S97209,ACTGCTAGCCAGTGTT
978,S95210-S97210,CATAACGGTGCACCAA
979,S95211-S97211,CAGTCTTCTTGACAGG
980,S95212-S97212,TGCCTCTTAGGCATAG
981,S95213-S97213,ACTGTGTCTAGCCGAA
982,S95214-S97214,GTATTGGCTTG